In [1]:
import cplex

# Boggle Board Generator

We have 16 squares, indexed by $j=0,...,15$.


| The grid: |     |     |     |     |
| ---       | --- | --- | ----| --- |
|           |  0  |  1  |  2  |  3  |
|           |  4  |  5  |  6  |  7  |
|           |  8  |  9  |  10 |  11 | 
|           | 12  |  13 |  14 |  15 |


We have the set of words that must appear $W$.  For some word $w \in W$, the length of word $w$ is $m_w$.  Let $i$ be the index of the $i^{th}$ letter of the word, where $i=0,...,m_w-1$.  Notationally we may refer to the letter as $w_i$.  The words are composed of the unique set of letters $A$, where $|A|=n$.  The maximum word length is 16.

Define the "decision variables" $x$ as
\begin{align}
x_{wij} &= \begin{cases} 1 & \mbox{ if word } w \mbox{ letter index } i \mbox{ is assigned to square } j,\\
                       0 & \mbox{ otherwise }
             \end{cases}
\end{align}

We also introduce an additional decision variable $y$, where
\begin{align}
y_{j} &= \begin{cases} 1 & \mbox{ if square } j \mbox{ is assigned "filler"},\\
                       0 & \mbox{ otherwise. }
        \end{cases}
\end{align}


It's worth noting here that we have $16 \times O(16) \times |W| + 16$ variables.  That means we have up to approximately $2^{256|W|}$ possible solutions.  Not all solutions will be valid.  If a solution satisfies the following constraints, then it is valid.

Only one letter or filler is assigned per square:
\begin{align}
y_j + \sum_{w \in W} \sum_{i=0}^{m_w} x_{wij} &= 1 && \forall j=0,...,15
\end{align}

Letter $w_i$ appears in at least one square (for each word $w$, each letter $w_i$).
\begin{align}
\sum_{j=0}^{15} x_{wij} &\ge 1 && \forall w \in W, & \forall i=0,...,m_w
\end{align}

Let $\Omega_j$ be the set of neighbors of square $j$ with respect to the grid indexes above.  For example:
\begin{align}
\Omega_0 &= \{ 1, 4, 5 \} \\
\Omega_5 &= \{ 0, 1, 2, 4, 6, 8, 9, 10 \}\\
\text{etc.}
\end{align}

For each word $w \in W$, consider the sequence of letters $w_0w_1...w_{m-1}$
\begin{align}
\sum_{\ell \in \Omega_j} x_{w (i+1) \ell} &\ge x_{wij} && \forall w \in W, & \forall i=0,...,m_w-1, &\forall j=0,...,15
\end{align}

Each word $w \in W$ must start in some square.
\begin{align}
\sum_{j=0}^{15} x_{w0j} &= 1 && \forall w \in W, & \forall i=0,...,m_w
\end{align}

In [2]:
word_list = ["gold","blue","red"]

filler_string = ""
alphabet = [filler_string] #always have some filler letter(s)

for w in word_list:
    for i in range(len(w)):
        if w[i] not in alphabet:
            alphabet.append(w[i])

print(alphabet)

['', 'g', 'o', 'l', 'd', 'b', 'u', 'e', 'r']


In [3]:
Omega = [] #init
Omega.append([1,4,5]) # 0
Omega.append([0,2,4,5,6]) # 1
Omega.append([1,3,5,6,7]) # 2
Omega.append([2,6,7]) # 3
Omega.append([0,1,5,8,9]) # 4
Omega.append([0,1,2,4,6,8,9,10]) # 5
Omega.append([1,2,3,5,7,9,10,11]) # 6
Omega.append([2,3,6,10,11]) # 7
Omega.append([4,5,9,12,13]) # 8
Omega.append([4,5,6,8,10,12,13,14]) # 9
Omega.append([5,6,7,9,11,13,14,15]) # 10
Omega.append([6,7,10,14,15]) # 11
Omega.append([8,9,13]) # 12
Omega.append([8,9,10,12,14]) # 13
Omega.append([9,10,11,13,15]) # 14
Omega.append([10,11,14]) # 15

for j in range(len(Omega)):
    print("Omega",j,":",Omega[j])

Omega 0 : [1, 4, 5]
Omega 1 : [0, 2, 4, 5, 6]
Omega 2 : [1, 3, 5, 6, 7]
Omega 3 : [2, 6, 7]
Omega 4 : [0, 1, 5, 8, 9]
Omega 5 : [0, 1, 2, 4, 6, 8, 9, 10]
Omega 6 : [1, 2, 3, 5, 7, 9, 10, 11]
Omega 7 : [2, 3, 6, 10, 11]
Omega 8 : [4, 5, 9, 12, 13]
Omega 9 : [4, 5, 6, 8, 10, 12, 13, 14]
Omega 10 : [5, 6, 7, 9, 11, 13, 14, 15]
Omega 11 : [6, 7, 10, 14, 15]
Omega 12 : [8, 9, 13]
Omega 13 : [8, 9, 10, 12, 14]
Omega 14 : [9, 10, 11, 13, 15]
Omega 15 : [10, 11, 14]


In [4]:
def x_variable_namer(word,letter_index,letter,square_index):
    return "x_" + word + "_" + str(letter_index) + "_" + letter + "_" + str(square_index) 

In [5]:
c = cplex.Cplex() #init model


c.set_problem_name("boggle_generator")


n = 16
m = len(alphabet)


# set objective to prefer the FILLER word.  Just to reduce the number of degenerate solutions.
c.objective.set_sense(c.objective.sense.maximize)


y_var_names = []
obj_coeffs = []
for j in range(n):
    y_var_names.append("y_" + str(j))
    obj_coeffs.append(1.0) #prefer filler
c.variables.add(names=y_var_names,
    types=[c.variables.type.binary]*n,
    obj=obj_coeffs)


x_var_names = []
obj_coeffs = []
for w in word_list:
    for i in range(len(w)):
        for j in range(n):
            x_var_names.append(x_variable_namer(w,i,w[i],j))
            obj_coeffs.append(0.0)
c.variables.add(names=x_var_names,
               types=[c.variables.type.binary]*len(x_var_names),
               obj=obj_coeffs)


# constraint: only one letter per square
for j in range(n):
    var_list =[]
    coeff_list = []
    var_list.append("y_" + str(j))
    coeff_list.append(1.0)
    for w in word_list:
        for i in range(len(w)):
            var_list.append(x_variable_namer(w,i,w[i],j))
            coeff_list.append(1.0)
    c.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=var_list, val=coeff_list)],
                            senses=["E"],
                            rhs=[1.0],
                            names=["only_one_letter_in_square_" + str(j)])


# constraint: letter w_i appears in at least one square
for w in word_list:
    for i in range(len(w)):
        var_list = []
        coeff_list = []
        for j in range(n):
            var_list.append(x_variable_namer(w,i,w[i],j))
            coeff_list.append(1.0)
        c.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=var_list, val=coeff_list)],
                             senses=["G"],
                             rhs=[1.0],
                             names=[w + "_" + str(i) + "_" + w[i] + "_must_appear_in_some_square"])


# constraint: consecutive letters need to appear within the omega neighborhood
for w in word_list:
    for i in range(len(w)-1):
        for j in range(n):
            var_list = []
            coeff_list = []
            var_list.append(x_variable_namer(w,i,w[i],j))
            coeff_list.append(-1.0)
            for ell in Omega[j]:
                var_list.append(x_variable_namer(w,i+1,w[i+1],ell))
                coeff_list.append(1.0)
            c.linear_constraints.add(lin_expr=[cplex.SparsePair(ind=var_list,val=coeff_list)],
                                    senses=["G"],
                                    rhs=[0.0],
                                    names=[w + "_" + w[i] + "_" + w[i+1] + "_neighborhood_of_square_" + str(j)])
            

    
# write the model to a file
c.write(c.get_problem_name() + ".lpt")

print(c.get_stats())

Problem name         : boggle_generator
Objective sense      : Maximize
Variables            :     192  [Binary: 192]
Objective nonzeros   :      16
Linear constraints   :     155  [Greater: 139,  Equal: 16]
  Nonzeros           :    1168
  RHS nonzeros       :      27

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzeros   : Min   : 1.000000         Max   : 1.000000       
Linear constraints   :
  Nonzeros           : Min   : 1.000000         Max   : 1.000000       
  RHS nonzeros       : Min   : 1.000000         Max   : 1.000000       



In [6]:
#Optional: read in problem from local file
#c.read("xxxxxx") 

c.solve()

Version identifier: 22.1.0.0 | 2022-03-25 | 54982fbec
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 155 rows, 192 columns, and 1168 nonzeros.
Reduced MIP has 192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.57 ticks)
Found incumbent of value 0.000000 after 0.01 sec. (1.78 ticks)
Probing time = 0.00 sec. (0.35 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 155 rows, 192 columns, and 1168 nonzeros.
Reduced MIP has 192 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.06 ticks)
Probing time = 0.00 sec. (0.35 ticks)
Clique table members: 16.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (0.57 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt  

In [7]:
var_names = c.variables.get_names()
solution_values = c.solution.get_values()

for v in range(len(var_names)):
    if abs(solution_values[v]) > 1e-9: #only print nonzero values
        print(var_names[v],":",solution_values[v])

y_1 : 1.0
y_8 : 1.0
y_11 : 1.0
y_12 : 1.0
y_13 : 1.0
x_gold_0_g_4 : 1.0
x_gold_1_o_9 : 1.0
x_gold_2_l_14 : 1.0
x_gold_3_d_15 : 1.0
x_blue_0_b_10 : 1.0
x_blue_1_l_6 : 1.0
x_blue_2_u_5 : 1.0
x_blue_3_e_0 : 1.0
x_red_0_r_2 : 1.0
x_red_1_e_7 : 1.0
x_red_2_d_3 : 1.0


In [8]:
grid = [[0,0,0,0], [0,0,0,0], [0,0,0,0], [0,0,0,0]]


for v in range(len(var_names)):
    if abs(solution_values[v]) > 1e-9:
        if var_names[v].split("_")[0] == "y":
            j = var_names[v].split("_")[1]
            fill = filler_string
        else:
            j = var_names[v].split("_")[-1]
            fill = var_names[v].split("_")[-2]
        
        row = int(j) // 4
        col = int(j) % 4
        grid[row][col] = fill
    
    
for row in grid:
    print(row)

['e', '', 'r', 'd']
['g', 'u', 'l', 'e']
['', 'o', 'b', '']
['', '', 'l', 'd']


# Summary


We gave an Integer Programming (IP) formulation for generating a Boggle board given a set of words.  We used IBM's CPLEX software to solve the IP.  If the list of words cannot be placed on a Boggle board, then the IP is "infeasible", else a solution is returned.  There may be more feasible solutions, but only one is returned.  We emphasize the use of "filler" characters to reduce the number of solutions.

Of note, if there is one feasible solution, then there are eight feasible solutions due to symmetry.  A Boggle board is a good example of dihedral group symmetry.  If the solution reported is $I$ (the identity), then we can perform a clockwise rotation of 90 degrees ($R$) on $I$, or multiple 90 degree rotations ($R^k$).  We note that $R^4=I$.  We can also reflect the Boggle board about the horizontal axis ($H$), the vertical axis ($V$), the diagonal axis ($D$), and the reverse diagonal axis ($D'$).  All of these operations still produce a valid Boggle board.  The full group multiplication table is summarized below.  We note that the group is non-Abelian.

\begin{array}{c|cccccccc}
    &  I  &  R  &  R^2  &  R^3  &  H  &  V  &  D  &  D' \\
\hline
I    &  I  &  R  &  R^2  &  R^3  &  H  &  V  &  D  &  D' \\
R    &  R  &  R^2&  R^3  &  I    &  D  &  D' &  V  &  H  \\
R^2  &  R^2&  R^3&  I    &  R    &  V  &  H  &  D' &  D  \\
R^3  &  R^3&  I  &  R    &  R^2  &  D' &  D  &  H  &  V  \\
H    &  H  &  D' &  V    &  D    &  I  &  R^2&  R^3&  R  \\
V    &  V  &  D  &  H    &  D'   &  R^2&  I  &  R  &  R^3\\
D    &  D  &  H  &  D'   &  V    &  R  &  R^3&  I  &  R^2\\
D'   &  D' &  V  &  D    &  H    &  R^3&  R  &  R^2 &  I \\
\end{array}
